Testing Playground :)

In [195]:
# %pip install onnx==1.12.0

In [196]:
import cv2
import torch
import numpy as np
import yaml
import os

In [205]:
opencv_net = cv2.dnn.readNet('yolov5/models/yolov5s.onnx')
imput_vid = cv2.VideoCapture('input/1.mp4')
filename = 'input/1.mp4'

classes = []

with open('yolov5/data/coco.yaml', 'r') as file:
        dict = yaml.load(file, Loader=yaml.FullLoader)
        arr = list(dict['names'].items())
        for key in range(len(arr)):
                classes.append(arr[key][1]) # coco 80 classes name

if (imput_vid.isOpened() == True):
        width = int(imput_vid.get(3))
        height = int(imput_vid.get(4))
        fps = imput_vid.get(5)
        print('Weight x Height FPS: ', width, 'x' ,height, fps)

        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        video = cv2.VideoWriter('output.avi', fourcc, fps, (width,height))
else:
        print('Can''t load the file, try another one :(')


# os.path.basename(filename).split(".")[0]


Weight x Height FPS:  1920 x 1080 29.97002997002997


In [206]:
def preprocess(input, net):
        blob = cv2.dnn.blobFromImage(input, 1/255, (640, 640), [0, 0, 0], True, crop=False)
        net.setInput(blob)
        output = net.forward(net.getUnconnectedOutLayersNames())

        # print('Preprocess done')
        return output

In [207]:
def postprocess(input, outputs):
        class_ids = []
        confidences = []
        boxes = []
        
        rows = outputs[0].shape[1]

        for r in range(rows):
                detections = outputs[0][0][r]
                confidence = detections[4]
                if confidence > 0.5:
                        scores = detections[5:]
                        class_id = np.argmax(scores)
                        center_x = int(detections[0] * width)
                        center_y = int(detections[1] * height)
                        w = int(detections[2] * width)
                        h = int(detections[3] * height)

                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) # (boxes, class scores, score threshold, nms threshold)
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                cv2.rectangle(input, (x, y), (x + w, y + h), (0,0,0), 3)
                cv2.putText(input, label, (x, y + 10), cv2.FONT_HERSHEY_PLAIN, 8, [0,0,0], 8)

        # print('Postprocess done')
        return input

In [200]:
while(imput_vid.isOpened()):
    flg, frames = imput_vid.read()
    if flg==True:
        input_blob = preprocess(frames, opencv_net)
        output = postprocess(frames, input_blob)
        video.write(output)
    else:
        break

In [201]:
imput_vid.release()
video.release()
cv2.destroyAllWindows()